In [1]:
load('../sim_data/independent_compLasso.RData')

In [2]:
dir = '../sim_data'
dim.list = list()
size = c(50, 100, 500, 1000)
idx = 0
for (P in size){
    for (N in size){
        idx = idx + 1
        dim.list[[idx]] = c(P=P, N=N)
    }
}

files = NULL
for (dim in dim.list){
    p = dim[1]
    n = dim[2]
    files = cbind(files, paste0(dir, '/sim_independent_', paste('P', p, 'N', n, sep='_'), '.RData'))
}

In [3]:
avg_FDR = NULL
for (i in 1:length(files)){
    sim_file = files[i]
    load(sim_file, dat <- new.env())
    sub = dat$sim_array[[i]]
    p = sub$p # take true values from 1st replicate of each simulated data
    coef = sub$beta
    coef.true = which(coef != 0)
    
    tt = results_ind_compLasso[[i]]$Stab.table
    
    FDR = NULL
    for (r in 1:nrow(tt)){
        FDR = c(FDR, length(setdiff(which(tt[r, ] !=0), coef.true))/sum(tt[r, ]))

    }
    
    avg_FDR = c(avg_FDR, mean(FDR, na.rm=T))
}

In [4]:
table_ind = NULL
tmp_num_select = rep(0, length(results_ind_compLasso))
for (i in 1:length(results_ind_compLasso)){
    table_ind = rbind(table_ind, results_ind_compLasso[[i]][c('n', 'p', 'rou', 'FP', 'FN', 'MSE', 'Stab')])
    tmp_num_select[i] = mean(rowSums(results_ind_compLasso[[i]]$Stab.table))
    
}
table_ind = as.data.frame(table_ind)
table_ind$num_select = tmp_num_select
table_ind$FDR = round(avg_FDR,2)

In [5]:
head(table_ind)

n,p,rou,FP,FN,MSE,Stab,num_select,FDR
50,50,0,2.99 ( 0.27 ),0.05 ( 0.02 ),0.8 ( 0.05 ),0.61,8.94,0.28
100,50,0,0.79 ( 0.15 ),0 ( 0 ),0.98 ( 0.06 ),0.87,6.79,0.09
500,50,0,0.51 ( 0.11 ),0 ( 0 ),1.08 ( 0.05 ),0.91,6.51,0.06
1000,50,0,0.39 ( 0.15 ),0 ( 0 ),1.18 ( 0.04 ),0.93,6.39,0.04
50,100,0,4.3 ( 0.31 ),0.24 ( 0.05 ),1.05 ( 0.07 ),0.52,10.06,0.37
100,100,0,1.02 ( 0.19 ),0 ( 0 ),0.94 ( 0.04 ),0.85,7.02,0.11


In [6]:
# export result
result.table_ind <- apply(table_ind,2,as.character)
rownames(result.table_ind) = rownames(table_ind)
result.table_ind = as.data.frame(result.table_ind)

# extract numbers only for 'n' & 'p'
result.table_ind$n = tidyr::extract_numeric(result.table_ind$n)
result.table_ind$p = tidyr::extract_numeric(result.table_ind$p)
result.table_ind$ratio = result.table_ind$p / result.table_ind$n

result.table_ind = result.table_ind[c('n', 'p', 'ratio', 'Stab', 'MSE', 'FP', 'FN', 'num_select', 'FDR')]
colnames(result.table_ind)[1:3] = c('N', 'P', 'Ratio')

extract_numeric() is deprecated: please use readr::parse_number() instead
extract_numeric() is deprecated: please use readr::parse_number() instead


In [7]:
# convert interested measurements to be numeric
result.table_ind$Stab = as.numeric(as.character(result.table_ind$Stab))
result.table_ind$MSE_mean = as.numeric(substr(result.table_ind$MSE, start=1, stop=4))
result.table_ind$FP_mean = as.numeric(substr(result.table_ind$FP, start=1, stop=4))
result.table_ind$FN_mean = as.numeric(substr(result.table_ind$FN, start=1, stop=4))
result.table_ind$FN_mean[is.na(result.table_ind$FN_mean)] = 0
result.table_ind$num_select = as.numeric(as.character(result.table_ind$num_select))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [8]:
# fix missing values
result.table_ind$MSE_mean[is.na(result.table_ind$MSE_mean)] = 1
result.table_ind

N,P,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
50,50,1.00,0.61,0.8 ( 0.05 ),2.99 ( 0.27 ),0.05 ( 0.02 ),8.94,0.28,0.80,2.99,0.05
100,50,0.50,0.87,0.98 ( 0.06 ),0.79 ( 0.15 ),0 ( 0 ),6.79,0.09,0.98,0.79,0.00
500,50,0.10,0.91,1.08 ( 0.05 ),0.51 ( 0.11 ),0 ( 0 ),6.51,0.06,1.08,0.51,0.00
1000,50,0.05,0.93,1.18 ( 0.04 ),0.39 ( 0.15 ),0 ( 0 ),6.39,0.04,1.18,0.39,0.00
50,100,2.00,0.52,1.05 ( 0.07 ),4.3 ( 0.31 ),0.24 ( 0.05 ),10.06,0.37,1.05,4.30,0.24
100,100,1.00,0.85,0.94 ( 0.04 ),1.02 ( 0.19 ),0 ( 0 ),7.02,0.11,0.94,1.02,0.00
500,100,0.20,0.96,1.16 ( 0.04 ),0.25 ( 0.09 ),0 ( 0 ),6.25,0.03,1.16,0.25,0.00
1000,100,0.10,0.89,1.04 ( 0.05 ),0.71 ( 0.15 ),0 ( 0 ),6.71,0.08,1.04,0.71,0.00
50,500,10.00,0.38,2.18 ( 0.13 ),5.53 ( 0.36 ),1.27 ( 0.11 ),10.26,0.48,2.18,5.53,1.27
100,500,5.00,0.75,1 ( 0.05 ),1.9 ( 0.29 ),0.03 ( 0.02 ),7.87,0.18,1.00,1.90,0.03


In [9]:
## export
write.table(result.table_ind, '../results_summary/sim_ind_compLasso.txt', sep='\t', row.names=F)

In [6]:
library(ggplot2)
library(gridExtra)
pdf('../figures_sim/figure_independent_compLasso.pdf', height=5, width=7)
result.table_ind$N = as.factor(result.table_ind$N)
fig_ind_stab = ggplot(result.table_ind, aes(x=P, y=Stab, color=N)) + geom_point() + ylab('Stability')
fig_ind_mse = ggplot(result.table_ind, aes(x=P, y=MSE_mean, color=N)) + geom_point() + ylab('MSE')
fig_ind_fp = ggplot(result.table_ind, aes(x=P, y=FP_mean, color=N)) + geom_point() + ylab('False Positives')
fig_ind_fn = ggplot(result.table_ind, aes(x=P, y=FN_mean, color=N)) + geom_point() + ylab('False Negatives')
grid.arrange(fig_ind_stab, fig_ind_mse, fig_ind_fp, fig_ind_fn, ncol=2, top='Independent_compLasso')
dev.off()

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Warning message:
“Removed 1 rows containing missing values (geom_point).”

pdf 
  2